In [2]:
import pandas as pd
import numpy as np
from plotnine import *
from babel.numbers import format_currency, format_percent
import math
pd.set_option('mode.chained_assignment', None)

In [5]:
base_folder = 'C:\\Users\\karim\\Documents\\westgate\\data\\muga\\'

### Loans

In [182]:
loans_if = pd.read_csv(base_folder + 'all_loans_if.csv')
loans_if['branch'] = 'if'

loans_pr = pd.read_csv(base_folder + 'all_loans_pr.csv')
loans_pr['branch'] = 'pr'

In [183]:
for df in [loans_if, loans_pr]:
    df.rename(columns={
        'Released Date': 'Released',
        'Principal Amount': 'Principal',
        'Loan Product': 'Product',
        'Loan Status Name': 'Status',
        'Balance Amount': 'Balance'
    }, inplace=True)

In [184]:
loans_if.head()

,Loan #,Loan Id,Loan Duration,Maturity Date,First Name,Last Name,Full Name,Released,Principal,Disbursed By,...,Total Penalty Paid,Pending Due,Pending Fees Due,Pending Interest Due,Pending Penalty Due,Pending Principal Due,Balance,Status,Product,branch
0,IF100000315478,6067651,4 Months,06/28/2024,Kyle,Motelago,Kyle Motelago,04/05/2024,400.0,Online Transfer,...,0.00,0.00,0.0,0.00,0.0,0.0,364.96,Current,IF-Installment loan,if
1,IF100000318606,6107360,4 Months,07/11/2024,Jean,Marie Hemsworth-klassen,Jean Marie Hemsworth-klassen,04/01/2024,350.0,Online Transfer,...,0.00,351.82,135.0,11.82,55.0,150.0,747.61,Defaulted,IF-Installment loan,if
2,IF100000318566,6107373,4 Months,08/21/2024,Stephane,D'astous,Stephane D'astous,04/01/2024,400.0,Online Transfer,...,0.00,156.95,48.0,3.95,55.0,50.0,1076.56,Defaulted,IF-Installment loan,if
3,IF100000318988,6107391,4 Months,07/26/2024,Mark Curtis,Moore,Mark Curtis Moore,04/01/2024,400.0,Online Transfer,...,55.39,0.00,48.0,3.95,55.0,50.0,0.00,Fully Paid,IF-Installment loan,if
4,IF100000318857,6107406,4 Months,07/17/2024,Shawna,Barnaby,Shawna Barnaby,04/01/2024,450.0,Online Transfer,...,0.00,458.76,216.0,17.76,0.0,225.0,917.50,Not Taken Up,IF-Installment loan,if


In [185]:
loans_if['release_date'] = pd.to_datetime(loans_if['Released'])

In [186]:
loans_pr['release_date'] = pd.to_datetime(loans_pr['Released'], dayfirst=True)

In [187]:
loans_df = pd.concat([loans_if, loans_pr])

In [188]:
loans_df.head()

,Loan #,Loan Id,Loan Duration,Maturity Date,First Name,Last Name,Full Name,Released,Principal,Disbursed By,...,Pending Due,Pending Fees Due,Pending Interest Due,Pending Penalty Due,Pending Principal Due,Balance,Status,Product,branch,release_date
0,IF100000315478,6067651,4 Months,06/28/2024,Kyle,Motelago,Kyle Motelago,04/05/2024,400.0,Online Transfer,...,0.00,0.0,0.00,0.0,0.0,364.96,Current,IF-Installment loan,if,2024-04-05
1,IF100000318606,6107360,4 Months,07/11/2024,Jean,Marie Hemsworth-klassen,Jean Marie Hemsworth-klassen,04/01/2024,350.0,Online Transfer,...,351.82,135.0,11.82,55.0,150.0,747.61,Defaulted,IF-Installment loan,if,2024-04-01
2,IF100000318566,6107373,4 Months,08/21/2024,Stephane,D'astous,Stephane D'astous,04/01/2024,400.0,Online Transfer,...,156.95,48.0,3.95,55.0,50.0,1076.56,Defaulted,IF-Installment loan,if,2024-04-01
3,IF100000318988,6107391,4 Months,07/26/2024,Mark Curtis,Moore,Mark Curtis Moore,04/01/2024,400.0,Online Transfer,...,0.00,48.0,3.95,55.0,50.0,0.00,Fully Paid,IF-Installment loan,if,2024-04-01
4,IF100000318857,6107406,4 Months,07/17/2024,Shawna,Barnaby,Shawna Barnaby,04/01/2024,450.0,Online Transfer,...,458.76,216.0,17.76,0.0,225.0,917.50,Not Taken Up,IF-Installment loan,if,2024-04-01


In [189]:
len(loans_df)

4818

In [193]:
counts_df = loans_df[['branch', 'Status']].value_counts().reset_index()
counts_df.pivot(
    index='branch',
    columns='Status',
    values='count'
)

Status,Arrears,Credit Counseling,Current,Defaulted,Denied,Due Today,Fully Paid,METCREDIT,MetCredit,Not Taken Up,Past Maturity,Processing,Restructured,Write-Off
branch,,,,,,,,,,,,,,
if,12.0,1.0,1060.0,50.0,NaN,305.0,53.0,1.0,NaN,523.0,NaN,40.0,1.0,3.0
pr,39.0,1.0,1323.0,71.0,3.0,375.0,55.0,NaN,1.0,855.0,1.0,40.0,NaN,5.0


In [194]:
loans_df['release_year'] = loans_df['release_date'].dt.year
loans_df['release_month'] = loans_df['release_date'].dt.month
loans_df['release_day'] = loans_df['release_date'].dt.day

In [196]:
loans_df[['release_year', 'release_month']].value_counts()

release_year  release_month
2024          4                2771
              5                2047
Name: count, dtype: int64

In [197]:
loans_disbursed_df = loans_df[~loans_df['Status'].isin(['Denied', 'Not Taken Up', 'Processing'])]

In [199]:
loans_disbursed_df['Loan #'].nunique(), len(loans_disbursed_df)

(3357, 3357)

In [234]:
loans_disbursed_df['muga'] = loans_disbursed_df['Product'].isin(['DataCollectionLoan', 
                                                                'DataCollectionLoan_AIapproved'])

In [200]:
loans_disbursed_df['Principal'].dtype

dtype('float64')

In [214]:
loans_disbursed_df.columns

Index(['Loan #', 'Loan Id', 'Loan Duration', 'Maturity Date', 'First Name',
       'Last Name', 'Full Name', 'Released', 'Principal', 'Disbursed By',
       'Paid Amount', 'Total Due Amount', 'Total Principal Balance',
       'Total Principal Paid', 'Total Fees Balance', 'Total Fees Paid',
       'Total Interest Balance', 'Total Interest Paid',
       'Total Penalty Balance', 'Total Penalty Paid', 'Pending Due',
       'Pending Fees Due', 'Pending Interest Due', 'Pending Penalty Due',
       'Pending Principal Due', 'Balance', 'Status', 'Product', 'branch',
       'release_date', 'release_year', 'release_month', 'release_day'],
      dtype='object')

### Repayments

In [201]:
repayments_if = pd.read_csv(base_folder + 'all_repayments_if.csv')

repayments_pr = pd.read_csv(base_folder + 'all_repayments_pr.csv')

#repayments_df = pd.concat([repayments_if, repayments_pr])

In [202]:
repayments_if.head()

,Approved By,Borrower #,Borrower Name,Collection Date,Loan #,First Name,Last Name,Loan Id,Loan Balance Amount,Loan Status,Maturity Date,Total Paid Amount,Loan Product
0,NaN,CID0000042141,Myriam Steiman,04/01/2024,IF10000061685,Myriam,Steiman,5414877,0.0,Fully Paid,04/15/2024,84.48,IF-Installment loan
1,NaN,CID00000217838,Tamara Iftody,04/01/2024,IF100000217841,Tamara,Iftody,5455546,0.0,Fully Paid,04/16/2024,93.98,IF-Installment loan
2,NaN,CID00000143224,Bola Chau,04/01/2024,IF100000283280,Bola,Chau,5527179,217.6,Defaulted,04/30/2024,0.00,IF-Installment loan
3,NaN,CID00000237421,Susan Diane Lomonte,04/01/2024,IF100000283553,Susan,Diane Lomonte,5542832,0.0,Fully Paid,04/01/2024,115.44,IF-Installment loan
4,NaN,CID00000277294,Michele Carangelo,04/01/2024,IF100000283796,Michele,Carangelo,5625249,0.0,Fully Paid,04/01/2024,217.36,IF-Installment loan


In [203]:
repayments_pr.head()

,Borrower #,Borrower Name,Collection Date,Loan #,Loan Id,Loan Balance Amount,Loan Product,Loan Status,Total Paid Amount,Repayment Id
0,3805201,Shawn Marcel Kirkness,01/04/2024,PR1014746,5450658,0.00,PrestoLoan,Fully Paid,101.13,40634067
1,CID00000010701,Christine Collison,01/04/2024,PR100000010701,5501260,0.00,PrestoLoan,Fully Paid,0.00,40634069
2,CID00000193353,Gerard Benedict Jaen Tadiosa,01/04/2024,PR1015100,5543157,0.00,PrestoLoan,Fully Paid,46.99,40634072
3,CID00000168130,Brittney Linda Marie Morin,01/04/2024,PR1015181,5581257,96.06,PrestoLoan,Current,96.03,40634073
4,CID2024287048178,Sheri Lynn Massi,01/04/2024,PR202428704817,5684094,0.00,PrestoLoan,Fully Paid,203.89,40634081


In [204]:
repayments_if['collection_date'] = pd.to_datetime(repayments_if['Collection Date'])

In [205]:
repayments_pr['collection_date'] = pd.to_datetime(repayments_pr['Collection Date'], dayfirst=True)

In [206]:
repayments_df = pd.concat([repayments_pr, repayments_if])

In [212]:
repayments_df.rename(columns = {
    'Total Paid Amount': 'Amount'
}, inplace=True)

In [207]:
repayments_df['collection_year'] = repayments_df['collection_date'].dt.year
repayments_df['collection_month'] = repayments_df['collection_date'].dt.month
repayments_df['collection_day'] = repayments_df['collection_date'].dt.day

In [208]:
repayments_df.columns

Index(['Borrower #', 'Borrower Name', 'Collection Date', 'Loan #', 'Loan Id',
       'Loan Balance Amount', 'Loan Product', 'Loan Status',
       'Total Paid Amount', 'Repayment Id', 'collection_date', 'Approved By',
       'First Name', 'Last Name', 'Maturity Date', 'collection_year',
       'collection_month', 'collection_day'],
      dtype='object')

### Merge

In [235]:
loans_main_cols = ['muga', 'Loan #', 'Loan Id', 'Released', 'release_date', 
                    'release_year', 'release_month', 'Principal', 
                    'Status', 'Balance']

In [210]:
repayments_main_cols = ['Loan #', 'Loan Id', 'Amount', 'collection_date', 'collection_year',
                        'collection_month', 'collection_day']

In [236]:
merged_df = loans_disbursed_df[loans_main_cols].merge(
    repayments_df[repayments_main_cols].rename(columns={'Loan Id': 'Loan # Repayment'}),
    how='right', 
    on = 'Loan #'
)

In [238]:
len(merged_df), len(repayments_df)

(23304, 23304)

In [237]:
merged_df['days_since_release'] = (merged_df['collection_date'] - merged_df['release_date']).dt.days

merged_df['weeks_since_release'] = np.floor(merged_df['days_since_release'] / 7.0)

In [239]:
merged_df.head()

,muga,Loan #,Loan Id,Released,release_date,release_year,release_month,Principal,Status,Balance,Loan # Repayment,Amount,collection_date,collection_year,collection_month,collection_day,days_since_release,weeks_since_release
0,NaN,PR1014746,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,5450658,101.13,2024-04-01,2024,4,1,NaN,NaN
1,NaN,PR100000010701,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,5501260,0.00,2024-04-01,2024,4,1,NaN,NaN
2,NaN,PR1015100,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,5543157,46.99,2024-04-01,2024,4,1,NaN,NaN
3,NaN,PR1015181,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,5581257,96.03,2024-04-01,2024,4,1,NaN,NaN
4,NaN,PR202428704817,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,5684094,203.89,2024-04-01,2024,4,1,NaN,NaN


### Analysis

In [228]:
loans_disbursed_df['Product'].value_counts()

Product
PrestoLoan                       1849
IF-Installment loan              1459
DataCollectionLoan                 13
IF- Correction loan.               10
DataCollectionLoan_AIapproved       9
Long Term- High Cost Credit.        8
Z-Correction Loan                   7
Presto_Collection                   2
Name: count, dtype: int64

In [242]:
disbursed_df = loans_disbursed_df.groupby(['release_year', 'release_month', 'muga']).agg({
    'Principal': 'sum',
    'Loan #': 'nunique'
}).reset_index().rename(columns={'Loan #': 'Loans_released'})

disbursed_df

,release_year,release_month,muga,Principal,Loans_released
0,2024,4,False,753826.05,1928
1,2024,4,True,4350.00,11
2,2024,5,False,553666.98,1407
3,2024,5,True,4125.00,11


In [244]:
collected_df = merged_df.groupby(['release_year', 'release_month', 'weeks_since_release', 'muga']).agg({
                                    'Amount': 'sum',
                                    'Loan #': 'nunique'
                                }).reset_index().rename(columns={'Loan #': 'Loans_collected'})
collected_df.sort_values(['release_year', 'release_month', 'weeks_since_release', 'muga'])

,release_year,release_month,weeks_since_release,muga,Amount,Loans_collected
0,2024.0,4.0,0.0,False,20137.39,148
1,2024.0,4.0,1.0,False,93686.61,1110
2,2024.0,4.0,1.0,True,290.94,5
3,2024.0,4.0,2.0,False,69497.19,906
4,2024.0,4.0,2.0,True,396.47,7
5,2024.0,4.0,3.0,False,81235.01,1067
6,2024.0,4.0,3.0,True,290.94,5
7,2024.0,4.0,4.0,False,45442.25,615
8,2024.0,4.0,4.0,True,456.85,7
9,2024.0,4.0,5.0,False,46197.15,586


In [245]:
analysis_df = collected_df.merge(
    disbursed_df,
    how = 'left',
    on = ['release_year', 'release_month', 'muga']
)

In [246]:
analysis_df

,release_year,release_month,weeks_since_release,muga,Amount,Loans_collected,Principal,Loans_released
0,2024.0,4.0,0.0,False,20137.39,148,753826.05,1928
1,2024.0,4.0,1.0,False,93686.61,1110,753826.05,1928
2,2024.0,4.0,1.0,True,290.94,5,4350.00,11
3,2024.0,4.0,2.0,False,69497.19,906,753826.05,1928
4,2024.0,4.0,2.0,True,396.47,7,4350.00,11
5,2024.0,4.0,3.0,False,81235.01,1067,753826.05,1928
6,2024.0,4.0,3.0,True,290.94,5,4350.00,11
7,2024.0,4.0,4.0,False,45442.25,615,753826.05,1928
8,2024.0,4.0,4.0,True,456.85,7,4350.00,11
9,2024.0,4.0,5.0,False,46197.15,586,753826.05,1928


In [257]:
analysis_cum_df = analysis_df.groupby(['muga', 'release_month', 'Principal', 
                                        'Loans_released', 'weeks_since_release'])[['Amount']].sum()\
                                        .groupby(level=0).cumsum().reset_index()

analysis_cum_df['principal_repaid_pct'] = analysis_cum_df['Amount'] / analysis_cum_df['Principal']

analysis_cum_df

,muga,release_month,Principal,Loans_released,weeks_since_release,Amount,principal_repaid_pct
0,False,4.0,753826.05,1928,0.0,20137.39,0.026714
1,False,4.0,753826.05,1928,1.0,113824.00,0.150995
2,False,4.0,753826.05,1928,2.0,183321.19,0.243188
3,False,4.0,753826.05,1928,3.0,264556.20,0.350951
4,False,4.0,753826.05,1928,4.0,309998.45,0.411233
5,False,4.0,753826.05,1928,5.0,356195.60,0.472517
6,False,4.0,753826.05,1928,6.0,373947.22,0.496066
7,False,4.0,753826.05,1928,7.0,378062.65,0.501525
8,False,5.0,553666.98,1407,0.0,386543.02,0.698151
9,False,5.0,553666.98,1407,1.0,432279.06,0.780756


### Debugging

In [224]:
loans_df[loans_df['Loan #']=='PR1014746']

,Loan #,Loan Id,Loan Duration,Maturity Date,First Name,Last Name,Full Name,Released,Principal,Disbursed By,...,Pending Penalty Due,Pending Principal Due,Balance,Status,Product,branch,release_date,release_year,release_month,release_day


In [115]:
loans_4 = merged_df.query('weeks_since_release==4')['Loan#']

In [120]:
merged_df[merged_df['Loan#'].isin(loans_4)].sort_values(['Loan#', 'collection_date'])[[
    'Loan#', 'Released', 'release_date', 'Principal', 'Status', 'collection_date', 'Amount', 'Balance'
]]

,Loan#,Released,release_date,Principal,Status,collection_date,Amount,Balance
21,IF100000325795,04/17/2024,2024-04-17,350,Current,2024-05-03,NaN,354.47
16,IF100000325795,04/17/2024,2024-04-17,350,Current,2024-05-17,NaN,354.47
25,IF100000IF100000118978,04/17/2024,2024-04-17,350,Current,2024-04-24,NaN,332.34
23,IF100000IF100000118978,04/17/2024,2024-04-17,350,Current,2024-05-01,NaN,332.34
20,IF100000IF100000118978,04/17/2024,2024-04-17,350,Current,2024-05-08,NaN,332.34
19,IF100000IF100000118978,04/17/2024,2024-04-17,350,Current,2024-05-15,NaN,332.34
15,IF100000IF100000118978,04/17/2024,2024-04-17,350,Current,2024-05-22,NaN,332.34
8,PR1017400,18/04/2024,2024-04-18,350.00,Current,2024-05-08,0.00,782.30
1,PR1017400,18/04/2024,2024-04-18,350.00,Current,2024-05-22,60.38,782.30
12,PR1017401,18/04/2024,2024-04-18,450.00,Current,2024-05-02,119.93,719.58
